In [1]:
from bs4 import BeautifulSoup
import requests
import re
import json
import time
from kafka import KafkaProducer , KafkaConsumer

In [2]:
TOPIC_Step1_NAME="raw_datas"
TOPIC_Step2_NAME="jobs_info"
KAFKA_SERVER="broker:9092"

In [3]:
producer = KafkaProducer(bootstrap_servers=KAFKA_SERVER,  value_serializer=lambda m: json.dumps(m).encode('utf-8') )
delay = 3   
x=0

In [4]:
consumer = KafkaConsumer(
     TOPIC_Step1_NAME,
     bootstrap_servers=[KAFKA_SERVER],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     group_id='raw_data',
     value_deserializer=lambda x: json.loads(x.decode('utf-8')),
     key_deserializer=lambda x : str(x)
)

In [5]:
def crawl_item(item_url,passed_day):
    
    res = requests.get(item_url)
    time.sleep(delay)
    soup = BeautifulSoup(res.content, 'html.parser')

    
    company_name = soup.find('h2' , {'class':'c-companyHeader__name'}).text

    title= soup.find('h1').text
    first = soup.find_all('span' , {'class':'black'})
    category = first[0].text
    location = first[1].text
    contract_type = first[2].text
    exprience = first[3].text
    salary = first[4].text

    work_details = soup.find('div' , {'class':'o-box__text'}).text

    processed_data={

        'company' : company_name,
        'job_title' : title,
        'job_category' : category,
        'job_location' : location,
        'contract_type' : contract_type,
        'experience' : exprience,
        'salary' : salary,
        'more_details' : work_details,
        'passed_day' : passed_day,
        'job_url' : item_url
    }
    print(processed_data)
    producer.send(TOPIC_Step2_NAME, processed_data)
    

In [ ]:
for message in consumer:
    try :
        data=message.value
        item_title = data['item-title']

#         if re.search(r'\bdata\b.*\bengineer\b|\bengineer\b.*\bdata\b|مهندس داده|کاراموز|کارآموز', item_title, re.IGNORECASE):
#             crawl_item(data['item_url'], data['passed_day'])


        if re.search(r'\bdata\b.*\bengineer\b|\bengineer\b.*\bdata\b|مهندس داده', item_title, re.IGNORECASE):
            crawl_item(data['item_url'], data['passed_day'])


            
    except Exception as ex:
        print("%%%-"*20)
        print(ex)

{'company': 'دان | Done', 'job_title': ' استخدام Data Engineer(مشهد)', 'job_category': 'وب،\u200c برنامه\u200cنویسی و نرم\u200cافزار', 'job_location': 'خراسان رضوی\n                                                ، مشهد', 'contract_type': '\n                                    تمام وقت\n                                    ', 'experience': 'مهم نیست', 'salary': 'توافقی', 'more_details': '\nطراحی زیرساخت مورد نیاز برای ETL بهینه، استخراج و انتقال ، بارگذاری و بارگیری داده ها از منابع مختلف داده مانند فایل\u200cهای لاگ، دیتابیس\u200cهای SQL\xa0نگهداری و بهینه سازی پایپ لاین های دیتا به منظور آماده سازی دیتای تمیز برای تحلیل.شناسایی، طراحی و اجرای پروسه\u200cها و فرایندهای داخلی ، خودکار سازی فرایندهای دستی و بهینه سازی تحویل داده ها، اصلاح ساختار دیتابیس ها با توجه به 3NF، طراحی و بهبود زیرساخت داده برای مقیاس پذیری بیشتر.جمع آوری مجموعه داده های بزرگ و پیچیده جهت الزامات تجاری و\xa0 اتصال زیرساخت داده به ابزارهای BIهمکاری با تیم\u200cها و ذینفعان مختلف مانند محصول، طراحی و تحلیل دیتا برا

{'company': 'اگزا | Exa DataScience Lab', 'job_title': ' استخدام Data Engineer', 'job_category': 'وب،\u200c برنامه\u200cنویسی و نرم\u200cافزار', 'job_location': 'تهران\n                                                ، تهران', 'contract_type': '\n                                    تمام وقت\n                                    ', 'experience': 'کمتر از سه سال', 'salary': 'توافقی', 'more_details': "\nResponsibilities:Develop data-driven applicationsDesign large scale Software for real-time and batch processingWork with structured, semi-structure and unstructured dataOptimization of the application for maximum speed and scalabilityProgramming in Python languageRequirements:Intermediate in Python Programming languageFamiliarity in Relational & NoSQL databasesHands-on experience with KafkaHands-on experience in ElasticsearchHands-on experience in Linux, DockerFamiliar with Agile cultureSoft skills:CommitmentCreativityTeam-workingSelf-managementHigh EnthusiasmProblem solvingFast LearningBen

{'company': 'اسنپ | Snapp', 'job_title': ' استخدام Senior Data Engineer', 'job_category': 'وب،\u200c برنامه\u200cنویسی و نرم\u200cافزار', 'job_location': 'تهران\n                                                ، تهران', 'contract_type': '\n                                    تمام وقت\n                                    ', 'experience': 'بیش از شش سال', 'salary': 'توافقی', 'more_details': '\nSnapp is the pioneer provider of ride-hailing mobile solutions in Iran that connects smartphone owners in need of a ride to Snapp drivers who use their private cars to offer transportation services. We are ambitious, passionate, engaged, and excited about pushing the boundaries of the transportation industry to new frontiers and being thefirst choice of each user in Iran.About the RoleThe role contribute to a variety of exciting projects ranging from designing robust and automated Data Pipelines and Storage processes to building tools for improving company-wide productivity with data. It’s more abo

{'company': 'رستم | Rostam', 'job_title': ' استخدام Data Engineer (دورکاری)', 'job_category': 'وب،\u200c برنامه\u200cنویسی و نرم\u200cافزار', 'job_location': 'تهران\n                                                ، تهران', 'contract_type': '\n                                    تمام وقت\n                                    ', 'experience': 'دورکاری', 'salary': 'سه تا شش سال', 'more_details': "\nWe are looking for an experienced Web Scraper Developer to join our fully remote team. In this role, you will be responsible for writing scrapers that use headless browsers, performing data cleaning tasks, and regularly uploading the data to our cloud storage. This role is critical in providing consistent, reliable, and high-quality data to our operations by running the scrapers on a strict schedule.\xa0\xa0Job Responsibilities:\xa01. Write efficient and robust scrapers using headless browsers.\xa0 2. Implement IP rotation to ensure the scrapers are not interrupted if needed.\xa0 3. Perform dat

{'company': 'اسنپ گروسری | Snapp Grocery', 'job_title': ' استخدام Data Engineer', 'job_category': 'وب،\u200c برنامه\u200cنویسی و نرم\u200cافزار', 'job_location': 'تهران\n                                                ، تهران', 'contract_type': '\n                                    تمام وقت\n                                    ', 'experience': 'سه تا شش سال', 'salary': 'توافقی', 'more_details': '\nAs a fast-growing e-commerce company, SnappGrocery strives to improve online shopping experiences. You will play an essential role in building and maintaining data systems. This job is for you if you are interested in combining data from different sources with great analytical skill.Responsibilities\xa0Design, optimal develop, and maintain standard data pipelines and according to data integration principles and business goals.Designing and implementing solutions for loading both structured and semi-structured data design into multiple target data systems.develop and test data systems to crea

{'company': 'پی پینگ | Payping', 'job_title': ' استخدام Data Engineer(مشهد)', 'job_category': 'IT / DevOps / Server', 'job_location': 'خراسان رضوی\n                                                ، مشهد', 'contract_type': '\n                                    تمام وقت\n                                    ', 'experience': 'کمتر از سه سال', 'salary': 'توافقی', 'more_details': '\nپی پینگ داره دنبال شما می گرده!فین تک یه حوزه به شدت داده\u200cمحوره و داشتن زیرساخت\u200cهای دیتابیسی و پیاده\u200cسازی Data Pipelines برای ما خیلی مهمه. برای همین دنبال یه هم\u200cتیمی جدید در سمت Data Engineerمی\u200cگردیم تا با طراحی و پیاده سازی فرایند های استخراج، ذخیره\u200cسازی و تبدیل داده، به ما در تولید و حفظ داده\u200cهای با کیفیت کمک کنه. اتوماتیک کردن این فرایند\u200cها موضوع مهمیه که شما در پیاده سازی اون به ما کمک می\u200cکنید. اگه این مهارت\u200cها رو دارید و برنامه\u200cنویسی با پایتون رو هم بلد هستید، جای شما همین جاست!رزومه\u200cتون رو برای ما بفرستید که خیلی مشتاقیم بیشتر باهم آشنا بشیم و یه

{'company': 'ابرناک | abrNOC', 'job_title': ' استخدام Data Center Network Engineer', 'job_category': 'IT / DevOps / Server', 'job_location': 'تهران\n                                                ، تهران', 'contract_type': '\n                                    تمام وقت\n                                    ', 'experience': 'سه تا شش سال', 'salary': 'از ۳۰,۰۰۰,۰۰۰ تومان', 'more_details': '\nما یک جای خالی در تیم ابرناک برای یک Network Engineer با تجربه که عاشق چالش و خلق کردن باشه، داریم.شرح وظایف:\xa0نظارت بر عملکرد شبکه و اطمینان از در دسترس بودن و مطمئن بودن سیستم\xa0نصب، پیکربندی و پشتیبانی تجهیزات شبکه از جمله روترها، سوئیچ ها، متعادل کننده های بار، دستگاه های VPN و فایروال ها\xa0پیاده سازی HA برای Router , Switch و Trafficتعریف،مستندسازی و اجرای استاندردهای سیستممدیریت ظرفیت آدرس های IP در مراکز داده\xa0حداقل های موردنیاز:\xa0حداقل مدرک کارشناسی در رشته علوم کامپیوتر یا موضوعات مرتبطتجربه عملی کار با ابزارهای نظارت، تشخیص شبکه و تجزیه و تحلیل شبکهتسلط کافی در ارتباط با DGPتجربه م

{'company': 'بازی گستران دریک | Bazigostaran Derick', 'job_title': ' استخدام Data Engineer', 'job_category': 'وب،\u200c برنامه\u200cنویسی و نرم\u200cافزار', 'job_location': 'تهران\n                                                ، تهران', 'contract_type': '\n                                    تمام وقت\n                                    ', 'experience': 'کمتر از سه سال', 'salary': 'توافقی', 'more_details': '\nما در استودیو مدریک به دنبال یک مهندس داده خلاق هستیم تا در طراحی و توسعه زیرساخت داده به ما کمک کند.\xa0وظایفطراحی و توسعه زیرساخت\u200cهای داده مورد نیاز برای تیم تحلیل دادهمانیتور کردن وضعیت پایپ\u200cلاین\u200cهای فعلیscale-up کردن پایپ\u200cلاین\u200cهای داده در صورت نیازمهارت\u200cهای مورد نیازمسلط به یکی از زبان\u200cهای برنامه\u200cنویسی Python، Java و Scalaمسلط به SQLمسلط به روش\u200cهای مدل\u200cسازی دادهآشنایی با الگو\u200c\u200cهای معماری نرم\u200c\u200cافزارتجربه کار با دیتابیس\u200cهای relational مانند Postgres و دیتابیس\u200cهای non-relational مانند Cassandra، Mon

{'company': 'شیوه نرم افزار گستر آسیا (سهامی خاص) | Shiveh Software Development Company', 'job_title': ' استخدام مهندس داده (junior)', 'job_category': 'وب،\u200c برنامه\u200cنویسی و نرم\u200cافزار', 'job_location': 'تهران\n                                                ، تهران', 'contract_type': '\n                                    تمام وقت\n                                    ', 'experience': 'کمتر از سه سال', 'salary': 'توافقی', 'more_details': '\nما\xa0 در مپ به دنبال یه مهندس داده ی جونیور هستیم که تیم داده نقشه مپ رو توسعه بدیم. اگر دوست\u200c دارید در کنار هم و به صورت تیمی کارهای جذاب و هیجان انگیز توی حوزه نقشه انجام بدیم و به درست کردن pipleline برای داده ها علاقه مند هستید، کافیه توانایی\u200cهای زیر رو داشته باشید:\xa0*\xa0 تسلط به بانک اطلاعاتی postgres* آشنایی با ساختمان داده ها و الگوریتم ها* آشنایی با زبان برنامه نویسی پایتون* آشنایی با ELK* آشنایی با Linux و docker* علاقه\u200cمند بودن به توسعه pipeline داده (airflow,nifi)حالا اگر از پس این کارها برمی\u200cآید و مدرک